### Come usare un notebook Jupyter

### Introduzione
Un notebook Jupyter è uno strumento interattivo che permette di scrivere e eseguire codice in vari linguaggi di programmazione, come Python, direttamente nel browser. È molto utile per analisi dati, machine learning, e visualizzazione dei dati.

### Struttura del notebook
Un notebook Jupyter è composto da celle. Ci sono due tipi principali di celle:
1. **Celle di codice**: dove si scrive ed esegue il codice.
2. **Celle di testo (Markdown)**: dove si può scrivere testo formattato usando Markdown.

### Come usare le celle
- **Aggiungere una cella**: Clicca sul pulsante `+` nella barra degli strumenti o usa il comando `Insert` dal menu.
- **Eseguire una cella**: Seleziona la cella e premi `Shift + Enter` oppure clicca sul pulsante `Run` nella barra degli strumenti.
- **Modificare una cella**: Clicca due volte sulla cella di testo o seleziona la cella di codice e inizia a scrivere.

Per utilizzare questo notebook, dovrai solo modificare i prompt nelle celle di codice e eseguire le celle per ottenere i risultati desiderati.

## init e dependencies

In [1]:
! pip install langchain==0.2.7 langchain_aws langchain-community langchain_core boto3 botocore

INFO: pip is looking at multiple versions of langchain-aws to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-aws to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 44.3 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
load_dotenv()
import os

### llm endpoints (to edit)

In [4]:
# from langchain_openai import AzureChatOpenAI

# llm = AzureChatOpenAI(
#     azure_deployment="gpt4o",
#     openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
#     temperature=0,
#     api_key=os.getenv("AZURE_OPENAI_API_KEY"),
#     azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
# )

In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [13]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name='eu-west-1',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        verify=False,  # Disable SSL verification
        config=Config(
            proxies={'https': None}
        )
    )

llm = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        client=bedrock_client,
        model_kwargs={
            "temperature": 0,
            "max_tokens": 2000,
        }
    )


## Esercitazioni

### 1. Basic prompt

In questa esercitazione, l'obiettivo è imparare a strutturare un prompt di base per generare una storia o una filastrocca in rima utilizzando l'llm fornito.
Questo sarà la nostra baseline per la valutazione dell'efficacia di prompt successivi.

#### Passaggi:

1. **Definizione del PromptTemplate**:
    - Utilizziamo la classe `PromptTemplate` per definire il template del prompt. Questo template include variabili di input come `protagonist` e `setting` che verranno sostituite con i valori forniti dall'utente.

     ```python
     prompt_template = PromptTemplate(
          input_variables=["protagonist", "setting"],
          template="""
                Genera una storia
                Protagonista:
                {protagonist} 

                Ambientazione:
                {setting}
          """
     )
     ```

2. **Creazione della catena di elaborazione**:
    - Combiniamo il `PromptTemplate` con il modello di linguaggio (`llm`) e un parser di output (`StrOutputParser`) per creare una catena di elaborazione (`chain`). Questa catena prende il prompt generato e lo invia al modello di linguaggio per ottenere una risposta.

     ```python
    chain = prompt_template | llm | StrOutputParser()
     ```

3. **Generazione della storia**:
    - Definiamo una funzione `generate` che prende in input il protagonista e l'ambientazione, e utilizza la catena di elaborazione per generare una storia.

     ```python
     def generate(protagonist, setting):
          response = chain.invoke({"protagonist": protagonist, "setting": setting})
          return response
     ```

4. **Esecuzione della funzione**:
    - Eseguiamo la funzione `generate` con i valori desiderati per il protagonista e l'ambientazione, e visualizziamo la storia generata.

     ```python
     response = generate("Alice", "un bosco")
     print(response)
     ```

Questa esercitazione mostra come utilizzare i componenti di LangChain per creare un prompt di base e generare contenuti creativi come storie o filastrocche in rima.

In [14]:
def generate(protagonist, setting):

    prompt_template = PromptTemplate(
        input_variables=["protagonist", "setting"],
        template="""
            Protagonista:
            {protagonist} 

            Ambientazione:
            {setting}
        """
    )

    chain = prompt_template | llm | StrOutputParser()

    response = chain.invoke({"protagonist": protagonist, "setting": setting})
    return response


In [15]:
response = generate("Alice", "un bosco")

In [16]:
response

"Ecco una possibile storia breve ambientata in un bosco con protagonista Alice:\n\nAlice si incamminò nel folto del bosco, le foglie secche scricchiolavano sotto i suoi passi. Aveva bisogno di staccarsi dalla routine quotidiana e trovare un po' di pace in mezzo alla natura. \n\nMentre camminava, il suo sguardo venne attratto da una luce soffusa che filtrava tra gli alberi. Incuriosita, si diresse in quella direzione e si ritrovò in una radura illuminata dai raggi del sole. Al centro c'era una vecchia quercia, i cui rami sembravano protendere verso di lei, come per accoglierla.\n\nAlice si sedette ai piedi dell'albero, appoggiando la schiena al tronco rugoso. Un senso di tranquillità la avvolse. Chiuse gli occhi, ascoltando il canto degli uccelli e il fruscio delle foglie mosse dal vento. In quel momento, ebbe l'impressione che la quercia stesse sussurrandole qualcosa. \n\nAprì gli occhi e sorrise, sentendosi finalmente in pace con se stessa. Il bosco le aveva regalato la serenità di cu

### 2. Few Shot Prompt

In questa esercitazione, l'obiettivo è imparare a creare un prompt "few-shot" per generare una storia o una filastrocca in rima di complessità superiore alla precedente.

#### Passaggi:

1. **Definizione del PromptTemplate**:
    - Utilizziamo la classe `PromptTemplate` per definire il template del prompt. Questo template include esempi di input e output per aiutare il modello a comprendere meglio il compito.
    - **Few-shot learning**: Il few-shot learning è una tecnica in cui il modello viene addestrato con un numero limitato di esempi (da pochi a poche decine) per ogni compito. Questo approccio è utile quando non si dispone di grandi quantità di dati di addestramento. Per costruire un prompt few-shot, includiamo nel template alcuni esempi di input e output che il modello può utilizzare come riferimento per generare risposte accurate.

2. **Creazione della catena di elaborazione**:
    - Combiniamo il `PromptTemplate` con il modello di linguaggio (`llm`) e un parser di output (`StrOutputParser`) per creare una catena di elaborazione (`chain`). Questa catena prende il prompt generato e lo invia al modello di linguaggio per ottenere una risposta.

3. **Generazione della storia**:
    - Definiamo una funzione `generate_few_shot` che prende in input il protagonista e l'ambientazione, e utilizza la catena di elaborazione per generare una storia.


4. **Esecuzione della funzione**:
    - Eseguiamo la funzione `generate_few_shot` con i valori desiderati per il protagonista e l'ambientazione, e visualizziamo la storia generata.


In [ ]:
def generate_few_shot(protagonist, setting):

    prompt_template = PromptTemplate(
        input_variables=["protagonist", "setting"],
        template="""
            Protagonista:
            {protagonist} 

            Ambientazione:
            {setting}
        """
    )

    chain = prompt_template | llm | StrOutputParser()

    response = chain.invoke({"protagonist": protagonist, "setting": setting})
    return response

### Guiding avanzato

In questa sezione, esploreremo come creare una catena di elaborazione avanzata utilizzando tecniche di "instruction tuning" e "role playing". Queste tecniche permettono di migliorare la qualità delle risposte generate dal modello di linguaggio, fornendo istruzioni dettagliate e simulando ruoli specifici.

***Difficoltà aggiuntiva***: cerchiamo di limitare il numero di parole generate dal modello.

#### Passaggi:

1. **Definizione del PromptTemplate**:
	- Utilizziamo la classe `PromptTemplate` per definire il template del prompt. Questo template include variabili di input come `protagonist` e `setting`, oltre a istruzioni dettagliate per il modello. L'instruction tuning consiste nel fornire al modello istruzioni chiare e dettagliate su come deve rispondere, mentre il role playing implica la simulazione di ruoli specifici per guidare il modello a generare risposte più contestualizzate.

2. **Creazione della catena di elaborazione**:
	- Combiniamo il `PromptTemplate` con il modello di linguaggio (`llm`) e un parser di output (`StrOutputParser`) per creare una catena di elaborazione (`chain`). Questa catena prende il prompt generato e lo invia al modello di linguaggio per ottenere una risposta.

3. **Generazione della storia**:
	- Definiamo una funzione `generate_guiding` che prende in input il protagonista e l'ambientazione, e utilizza la catena di elaborazione per generare una storia.

4. **Esecuzione della funzione**:
	- Eseguiamo la funzione `generate_guiding` con i valori desiderati per il protagonista e l'ambientazione, e visualizziamo la storia generata.
    #### Risultato:

    La funzione `generate_guiding` utilizza tecniche di instruction tuning e role playing per generare una storia più dettagliata e contestualizzata. Ecco un esempio di storia generata:



In [ ]:
response = generate_few_shot("Alice", "un bosco")

In [ ]:
response

In [ ]:
def generate_guiding(protagonist, setting):

    prompt_template = PromptTemplate(
        input_variables=["protagonist", "setting"],
        template="""
            Protagonista:
            {protagonist} 

            Ambientazione:
            {setting}
        """
    )

    chain = prompt_template | llm | StrOutputParser()

    response = chain.invoke({"protagonist": protagonist, "setting": setting})
    return response

In [ ]:
response = generate_guiding("Alice", "un bosco")

In [ ]:
response